In [1]:
import numpy as np
import math
import time
import pandas as pd
from itertools import combinations, product, permutations
import random
from tqdm import tqdm

In [2]:
def epsilon_1(wavenumbers, w_0, w_p, gamma):
    return pow(w_p,2)*(w_0**2 - wavenumbers**2)/(( pow(wavenumbers,2) - pow(w_0,2) )**2+(w_0*gamma)**2)

def epsilon_2( wavenumbers, w_0, w_p, gamma ):
    return wavenumbers * gamma * pow(w_p,2) / ( (w_0*gamma)**2 + ( pow(wavenumbers,2) - pow(w_0,2) )**2)

def multi_epsilon_1(wavenumbers, params ):
    paramsRest = params[1:] # params[j][0] #e_inf
    e_inf=params[0]
    return e_inf + epsilon_1( wavenumbers, paramsRest[0], paramsRest[1][0], paramsRest[1][1] ) + epsilon_1( wavenumbers, paramsRest[2], paramsRest[3][0], paramsRest[3][1] ) 

def multi_epsilon_2( wavenumbers, params ):
    paramsRest = params[1:]
    return epsilon_2( wavenumbers, paramsRest[0], paramsRest[1][0], paramsRest[1][1] ) + epsilon_2( wavenumbers, paramsRest[2], paramsRest[3][0], paramsRest[3][1] )   
def real_refractive( wavenumbers, params):
    e1=np.asarray(multi_epsilon_1(wavenumbers, params))
    e2=np.asarray(multi_epsilon_2(wavenumbers, params))
    return np.sqrt(e1 + np.sqrt(e1*e1 + e2*e2) )/np.sqrt(2)

def imagi_refractive( wavenumbers, params):
    e1=np.asarray(multi_epsilon_1(wavenumbers, params))
    e2=np.asarray(multi_epsilon_2(wavenumbers, params))
    return e2/np.sqrt(2*e1 + 2*np.sqrt(e1*e1 + e2 * e2))

In [3]:
k=np.linspace(1000,8000,3501) #wavenumber range
w_p=np.linspace(0,1000,11) #plasma frequency range(0,100,200,300...,1000)
gam=np.arange(1,33,8) #gamma range (1,9,17,25)
e_range=np.arange(1.5, 2.5, 0.1) #epsilon_inf range
testData=[]
count=0
parameters=list(product(w_p,gam))
peak_par=list(permutations(parameters,2))
n_spectra=[]
ka_spectra=[]

In [4]:
set_param=[0,1100,0,1250,0]
start=time.time()

In [5]:
for i in tqdm(range(len(e_range))):
    for j in range(len(peak_par)):
        set_param[0]=e_range[i]
        set_param[2]=peak_par[j][0]
        set_param[4]=peak_par[j][1]
        
        n_spectra.append(real_refractive(k, set_param)) #epsilon_1 spectrum with 3 peak position
        ka_spectra.append(imagi_refractive(k, set_param)) #epsilon_2 spectrum with 3 peak position
print('Running_Time:',time.time()-start)

100%|██████████| 10/10 [00:03<00:00,  2.76it/s]

Running_Time: 5.321347713470459


In [13]:
n_spectra=np.array(n_spectra)
ka_spectra=np.array(ka_spectra)

In [15]:
df_n_spc=pd.DataFrame(data=n_spectra.T, index=None)
df_ka_spc=pd.DataFrame(data=ka_spectra.T, index=None)

In [17]:
df_k_spc=pd.DataFrame(data=k, columns=['Wavenumbers'])

In [19]:
df_n_spc=pd.concat([df_k_spc, df_n_spc], axis=1)
df_ka_spc=pd.concat([df_k_spc, df_ka_spc], axis=1)

In [20]:
df_n_spc.tail(10)

,Wavenumbers,0,1,2,3,4,5,6,7,8,...,10550,10551,10552,10553,10554,10555,10556,10557,10558,10559
3491,7982.0,1.224745,1.224745,1.224679,1.224679,1.224679,1.224482,1.224482,1.224482,1.224154,...,1.541466,1.541466,1.540683,1.540683,1.540683,1.539795,1.539795,1.539795,1.538802,1.538802
3492,7984.0,1.224745,1.224745,1.224679,1.224679,1.224679,1.224482,1.224482,1.224482,1.224154,...,1.541470,1.541470,1.540687,1.540687,1.540687,1.539799,1.539799,1.539799,1.538807,1.538807
3493,7986.0,1.224745,1.224745,1.224679,1.224679,1.224679,1.224482,1.224482,1.224482,1.224154,...,1.541474,1.541474,1.540691,1.540691,1.540691,1.539804,1.539804,1.539804,1.538812,1.538812
3494,7988.0,1.224745,1.224745,1.224679,1.224679,1.224679,1.224482,1.224482,1.224482,1.224154,...,1.541478,1.541478,1.540696,1.540696,1.540696,1.539809,1.539809,1.539809,1.538818,1.538818
3495,7990.0,1.224745,1.224745,1.224679,1.224679,1.224679,1.224483,1.224483,1.224483,1.224155,...,1.541482,1.541482,1.540700,1.540700,1.540700,1.539814,1.539814,1.539814,1.538823,1.538823
3496,7992.0,1.224745,1.224745,1.224679,1.224679,1.224679,1.224483,1.224483,1.224483,1.224155,...,1.541485,1.541485,1.540704,1.540704,1.540704,1.539819,1.539819,1.539819,1.538828,1.538828
3497,7994.0,1.224745,1.224745,1.224679,1.224679,1.224679,1.224483,1.224483,1.224483,1.224155,...,1.541489,1.541489,1.540709,1.540709,1.540709,1.539824,1.539824,1.539824,1.538834,1.538834
3498,7996.0,1.224745,1.224745,1.224679,1.224679,1.224679,1.224483,1.224483,1.224483,1.224156,...,1.541493,1.541493,1.540713,1.540713,1.540713,1.539828,1.539828,1.539828,1.538839,1.538839
3499,7998.0,1.224745,1.224745,1.224679,1.224679,1.224679,1.224483,1.224483,1.224483,1.224156,...,1.541497,1.541497,1.540717,1.540717,1.540717,1.539833,1.539833,1.539833,1.538844,1.538844
3500,8000.0,1.224745,1.224745,1.224679,1.224679,1.224679,1.224483,1.224483,1.224483,1.224156,...,1.541501,1.541501,1.540722,1.540722,1.540722,1.539838,1.539838,1.539838,1.538850,1.538850


In [21]:
df_n_spc.to_csv('refractive_index\combination_n.csv')
df_ka_spc.to_csv('refractive_index\combination_k.csv')